# Phase 3: Data Preprocessing & Feature Engineering
###### This notebook focuses on transforming the cleaned transactional data into customer-level features suitable for machine learning. Key steps include defining the churn window, calculating RFM metrics, deriving customer tenure, and creating the churn target variable.

In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import seaborn as sns

# Set display options for better viewing of DataFrames
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Load Cleaned Data

In [2]:
cleaned_data_path = r'../data/processed/online_retail_cleaned.csv'

try:
    df_cleaned = pd.read_csv(cleaned_data_path)
    # Ensure 'InvoiceDate' is in datetime format, as it's crucial for time-based features
    df_cleaned['InvoiceDate'] = pd.to_datetime(df_cleaned['InvoiceDate'])
    print(f"\nSuccessfully loaded cleaned data from {cleaned_data_path}")
    print(f"Cleaned data shape: {df_cleaned.shape}")
    print(df_cleaned.info())
except FileNotFoundError:
    print(f"Error: Cleaned data file not found at {cleaned_data_path}. Please ensure 01_data_acquisition_and_eda.ipynb was run successfully.")
    df_cleaned = pd.DataFrame()
except Exception as e:
    print(f"An error occurred while loading cleaned data: {e}")
    df_cleaned = pd.DataFrame()


Successfully loaded cleaned data from ../data/processed/online_retail_cleaned.csv
Cleaned data shape: (779425, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 779425 entries, 0 to 779424
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Invoice      779425 non-null  int64         
 1   StockCode    779425 non-null  object        
 2   Description  779425 non-null  object        
 3   Quantity     779425 non-null  int64         
 4   InvoiceDate  779425 non-null  datetime64[ns]
 5   Price        779425 non-null  float64       
 6   Customer ID  779425 non-null  int64         
 7   Country      779425 non-null  object        
 8   TotalPrice   779425 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(3)
memory usage: 53.5+ MB
None


## 2. Define Observation and Churn Periods

In [3]:
# Based on the EDA, the data ends on 2011-12-09.
# We will define a 3-month churn window.

analysis_end_date = df_cleaned['InvoiceDate'].max() # The very last transaction date in our dataset
churn_window_duration = timedelta(days=90) # 3 months for the churn observation period
observation_end_date = analysis_end_date - churn_window_duration

print(f"\nAnalysis End Date (Last transaction in data): {analysis_end_date}")
print(f"Observation End Date (Features calculated up to this date): {observation_end_date}")
print(f"Churn Window (Period to check for churn): {observation_end_date} to {analysis_end_date}")


Analysis End Date (Last transaction in data): 2011-12-09 12:50:00
Observation End Date (Features calculated up to this date): 2011-09-10 12:50:00
Churn Window (Period to check for churn): 2011-09-10 12:50:00 to 2011-12-09 12:50:00


In [4]:
# Create a DataFrame for customers active up to the observation_end_date
df_observation = df_cleaned[df_cleaned['InvoiceDate'] <= observation_end_date].copy()
print(f"\nShape of data used for observation features: {df_observation.shape}")
if df_observation.empty:
    print("Warning: df_observation is empty. This will lead to empty customer features.")
    # We'll exit gracefully if df_observation is empty, as no features can be built.
    # This will prevent the KeyError later.


Shape of data used for observation features: (620572, 9)


## 3. Feature Engineering: RFM (Recency, Frequency, Monetary)
###### RFM is calculated only on the observation period to avoid data leakage from the churn window.

In [5]:
# Monetary Value (M): Sum of TotalPrice for each customer in the observation period  
# Group by 'Customer ID' and sum 'TotalPrice'
customer_monetary = df_observation.groupby('Customer ID')['TotalPrice'].sum().reset_index()
customer_monetary.rename(columns={'TotalPrice': 'Monetary'}, inplace=True)
print("\n--- Diagnostic: customer_monetary head ---")
customer_monetary.head()


--- Diagnostic: customer_monetary head ---


,Customer ID,Monetary
0,12346,77556.46
1,12347,3402.39
2,12348,1709.40
3,12349,2671.14
4,12350,334.40


In [6]:
# Frequency (F): Number of unique invoices for each customer in the observation period
customer_frequency = df_observation.groupby('Customer ID')['Invoice'].nunique().reset_index()
customer_frequency.rename(columns={'Invoice': 'Frequency'}, inplace=True)
print("\n--- Diagnostic: customer_frequency details ---")
print(f"Shape: {customer_frequency.shape}")
print(f"Columns: {customer_frequency.columns.tolist()}")
customer_frequency.head()


--- Diagnostic: customer_frequency details ---
Shape: (5281, 2)
Columns: ['Customer ID', 'Frequency']


,Customer ID,Frequency
0,12346,12
1,12347,6
2,12348,4
3,12349,3
4,12350,1


In [7]:
# Recency (R): Days since last purchase relative to the observation_end_date
# Find the last purchase date for each customer within the observation period
customer_last_purchase = df_observation.groupby('Customer ID')['InvoiceDate'].max().reset_index()
customer_last_purchase.rename(columns={'InvoiceDate': 'LastPurchaseDate'}, inplace=True)
# Calculate Recency: difference in days between observation_end_date and LastPurchaseDate
customer_last_purchase['Recency'] = (observation_end_date - customer_last_purchase['LastPurchaseDate']).dt.days
print("\n--- Diagnostic: customer_last_purchase head ---")
customer_last_purchase.head()


--- Diagnostic: customer_last_purchase head ---


,Customer ID,LastPurchaseDate,Recency
0,12346,2011-01-18 10:01:00,235
1,12347,2011-08-02 08:48:00,39
2,12348,2011-04-05 10:47:00,158
3,12349,2010-10-28 08:23:00,317
4,12350,2011-02-02 16:01:00,219


In [8]:
# Merge RFM features into a single DataFrame
customer_features = customer_monetary.merge(customer_frequency, on='Customer ID', how='inner')
customer_features = customer_features.merge(customer_last_purchase[['Customer ID', 'Recency']], on='Customer ID')

In [9]:
print("\n--- RFM Features (first 5 rows) ---")
customer_features.head()


--- RFM Features (first 5 rows) ---


,Customer ID,Monetary,Frequency,Recency
0,12346,77556.46,12,235
1,12347,3402.39,6,39
2,12348,1709.40,4,158
3,12349,2671.14,3,317
4,12350,334.40,1,219


In [10]:
customer_features.describe()

,Customer ID,Monetary,Frequency,Recency
count,5281.000000,5281.000000,5281.000000,5281.000000
mean,15324.100170,2637.863286,5.745692,207.282901
std,1712.323865,12225.544955,11.464582,175.007174
min,12346.000000,2.900000,1.000000,0.000000
25%,13850.000000,318.240000,1.000000,49.000000
50%,15313.000000,783.580000,3.000000,164.000000
75%,16807.000000,2078.540000,6.000000,326.000000
max,18287.000000,456780.490000,309.000000,648.000000


In [11]:
print(f"Shape after initial RFM merge: {customer_features.shape}")

Shape after initial RFM merge: (5281, 4)


## 4. Feature Engineering: Customer Tenure

In [12]:
# Tenure: Days since the customer's first purchase up to the observation_end_date.
# We'll use the minimum InvoiceDate as the registration date for each customer.

# Find the first purchase date for each customer in the *entire* cleaned dataset
# (as this is their true "registration" date)
customer_first_purchase = df_cleaned.groupby('Customer ID')['InvoiceDate'].min().reset_index()
customer_first_purchase.rename(columns={'InvoiceDate': 'FirstPurchaseDate'}, inplace=True)
print("\n--- Diagnostic: customer_first_purchase head ---")
customer_first_purchase.head()


--- Diagnostic: customer_first_purchase head ---


,Customer ID,FirstPurchaseDate
0,12346,2009-12-14 08:34:00
1,12347,2010-10-31 14:20:00
2,12348,2010-09-27 14:59:00
3,12349,2010-04-29 13:20:00
4,12350,2011-02-02 16:01:00


In [13]:
# Calculate Tenure relative to observation_end_date
customer_first_purchase['Tenure'] = (observation_end_date - customer_first_purchase['FirstPurchaseDate']).dt.days

In [14]:
# Merge Tenure into the customer_features DataFrame
customer_features = customer_features.merge(customer_first_purchase[['Customer ID', 'Tenure']], on='Customer ID',how='inner')
print("\n--- RFM and Tenure Features (first 5 rows) ---")
customer_features.head()


--- RFM and Tenure Features (first 5 rows) ---


,Customer ID,Monetary,Frequency,Recency,Tenure
0,12346,77556.46,12,235,635
1,12347,3402.39,6,39,313
2,12348,1709.40,4,158,347
3,12349,2671.14,3,317,498
4,12350,334.40,1,219,219


In [15]:
customer_features.describe()

,Customer ID,Monetary,Frequency,Recency,Tenure
count,5281.000000,5281.000000,5281.000000,5281.000000,5281.000000
mean,15324.100170,2637.863286,5.745692,207.282901,431.772770
std,1712.323865,12225.544955,11.464582,175.007174,180.518152
min,12346.000000,2.900000,1.000000,0.000000,0.000000
25%,13850.000000,318.240000,1.000000,49.000000,311.000000
50%,15313.000000,783.580000,3.000000,164.000000,474.000000
75%,16807.000000,2078.540000,6.000000,326.000000,587.000000
max,18287.000000,456780.490000,309.000000,648.000000,648.000000


In [16]:
print(f"Shape after adding Tenure: {customer_features.shape}")

Shape after adding Tenure: (5281, 5)


## 5. Create Churn Label (Target Variable)

In [17]:
# A customer is churned if they had activity *before* observation_end_date
# but *no activity* in the churn window (from observation_end_date to analysis_end_date).

# Get customers who made purchases in the churn window
customers_in_churn_window = df_cleaned[
    (df_cleaned['InvoiceDate'] > observation_end_date) &
    (df_cleaned['InvoiceDate'] <= analysis_end_date)
]['Customer ID'].unique()

In [18]:
# Create 'is_churned' column:
# Initialize all customers (from observation period) as potentially churned (1)
customer_features['is_churned'] = 1
# Mark customers who purchased in the churn window as NOT churned (0)
customer_features.loc[customer_features['Customer ID'].isin(customers_in_churn_window), 'is_churned'] = 0

In [19]:
print("\n--- Churn Label Distribution ---")
customer_features['is_churned'].value_counts()


--- Churn Label Distribution ---


is_churned
1    2989
0    2292
Name: count, dtype: int64

In [20]:
print(f"Churn rate: {customer_features['is_churned'].mean() * 100:.2f}%")

Churn rate: 56.60%


## 6. Add Other Customer-Level Features (e.g., Country)

In [21]:
# Get the most frequent country for each customer (assuming primary country of residence)
customer_country = df_cleaned.groupby('Customer ID')['Country'].agg(lambda x: x.mode()[0]).reset_index()
customer_country.rename(columns={'Country': 'PrimaryCountry'}, inplace=True)
print("\n--- Diagnostic: customer_country head ---")
customer_country.head()


--- Diagnostic: customer_country head ---


,Customer ID,PrimaryCountry
0,12346,United Kingdom
1,12347,Iceland
2,12348,Finland
3,12349,Italy
4,12350,Norway


In [22]:
customer_features = customer_features.merge(customer_country, on='Customer ID', how='left')

In [23]:
print("\n--- Customer Features with Primary Country (first 5 rows) ---")
customer_features.head()


--- Customer Features with Primary Country (first 5 rows) ---


,Customer ID,Monetary,Frequency,Recency,Tenure,is_churned,PrimaryCountry
0,12346,77556.46,12,235,635,1,United Kingdom
1,12347,3402.39,6,39,313,0,Iceland
2,12348,1709.40,4,158,347,0,Finland
3,12349,2671.14,3,317,498,0,Italy
4,12350,334.40,1,219,219,1,Norway


In [24]:
print("\n--- Top 10 Primary Countries in Feature Set ---")
customer_features['PrimaryCountry'].value_counts().head(10)


--- Top 10 Primary Countries in Feature Set ---


PrimaryCountry
United Kingdom    4814
Germany             92
France              78
Spain               33
Belgium             27
Netherlands         22
Portugal            20
Switzerland         20
Sweden              19
Australia           14
Name: count, dtype: int64

In [25]:
print(f"Shape after adding Primary Country: {customer_features.shape}")

Shape after adding Primary Country: (5281, 7)


## 7. Handle Categorical Features (One-Hot Encoding for 'PrimaryCountry')

In [26]:
# One-hot encode 'PrimaryCountry'. Be mindful of potentially too many unique countries.
# For simplicity, we can encode the top N countries and group others into 'Other'.

num_top_countries = 10 # You can adjust this number
top_countries = customer_features['PrimaryCountry'].value_counts().head(num_top_countries).index.tolist()

# Replace countries not in top_countries with 'Other'
customer_features['PrimaryCountry_Grouped'] = customer_features['PrimaryCountry'].apply(
    lambda x: x if x in top_countries else 'Other'
)

# Perform one-hot encoding
df_final_features = pd.get_dummies(customer_features, columns=['PrimaryCountry_Grouped'], prefix='Country', drop_first=True)
# drop_first=True avoids multicollinearity for linear models.

# Drop the original 'PrimaryCountry' column as it's now encoded
df_final_features.drop('PrimaryCountry', axis=1, inplace=True)

In [27]:
print("\n--- Final Feature Set (first 5 rows with one-hot encoded countries) ---")
df_final_features.head()


--- Final Feature Set (first 5 rows with one-hot encoded countries) ---


,Customer ID,Monetary,Frequency,Recency,Tenure,is_churned,Country_Belgium,Country_France,Country_Germany,Country_Netherlands,Country_Other,Country_Portugal,Country_Spain,Country_Sweden,Country_Switzerland,Country_United Kingdom
0,12346,77556.46,12,235,635,1,False,False,False,False,False,False,False,False,False,True
1,12347,3402.39,6,39,313,0,False,False,False,False,True,False,False,False,False,False
2,12348,1709.40,4,158,347,0,False,False,False,False,True,False,False,False,False,False
3,12349,2671.14,3,317,498,0,False,False,False,False,True,False,False,False,False,False
4,12350,334.40,1,219,219,1,False,False,False,False,True,False,False,False,False,False


In [28]:
print(f"Final feature set shape: {df_final_features.shape}")
df_final_features.info()

Final feature set shape: (5281, 16)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5281 entries, 0 to 5280
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Customer ID             5281 non-null   int64  
 1   Monetary                5281 non-null   float64
 2   Frequency               5281 non-null   int64  
 3   Recency                 5281 non-null   int64  
 4   Tenure                  5281 non-null   int64  
 5   is_churned              5281 non-null   int64  
 6   Country_Belgium         5281 non-null   bool   
 7   Country_France          5281 non-null   bool   
 8   Country_Germany         5281 non-null   bool   
 9   Country_Netherlands     5281 non-null   bool   
 10  Country_Other           5281 non-null   bool   
 11  Country_Portugal        5281 non-null   bool   
 12  Country_Spain           5281 non-null   bool   
 13  Country_Sweden          5281 non-null   bool   
 14  Coun

In [29]:
required_core_features = ['Recency', 'Frequency', 'Monetary', 'Tenure', 'is_churned']
missing_before_save = [col for col in required_core_features if col not in df_final_features.columns]
if missing_before_save:
    print(f"\nCRITICAL ERROR: The following core features are missing from df_final_features before saving: {missing_before_save}")
else:
    print("\nAll core RFM, Tenure, and churn features are present in df_final_features before saving.")


All core RFM, Tenure, and churn features are present in df_final_features before saving.


##  8. Save the Feature-Engineered Data

In [30]:
# Save the final DataFrame to the 'data/processed/' directory.
output_file_path = '../data/processed/customer_churn_features.csv'
df_final_features.to_csv(output_file_path, index=False)
print(f"\nFeature-engineered data saved to {output_file_path}")

print("\n--- Feature Engineering Complete ---")


Feature-engineered data saved to ../data/processed/customer_churn_features.csv

--- Feature Engineering Complete ---
